In [1]:
import pandas as pd
import numpy as np
import gc

from rec_lib.utils import reduce_mem_usage, set_target, customers_prep, articles_prep, df_lvl2_prep, top_12_recs
from rec_lib.utils import get_features, get_preds_result, col_convert_int
from rec_lib.models import get_lvl2_model_preds, get_lvl2_LGBM_model_preds
from rec_lib.metrics import precision_at_k, ap_k, recall, recall_at_k

import warnings
warnings.filterwarnings("ignore")

In [2]:
%load_ext autoreload
%autoreload 2

### Загрузка данных

In [3]:
%%time
result = pd.read_parquet('archive/result.parquet')
customers = pd.read_parquet('archive/customers.parquet')
articles = pd.read_parquet('archive/articles.parquet')
transactions = pd.read_parquet('archive/transactions_train_for_power_bi.parquet')

CPU times: user 7.39 s, sys: 2.61 s, total: 10 s
Wall time: 5.38 s


### Подготовка данных

In [4]:
%%time
# преобразование датафрейма пользователей
customers = customers_prep(customers)

CPU times: user 1.65 s, sys: 139 ms, total: 1.79 s
Wall time: 1.77 s


In [5]:
%%time
# преобразование датафрейма категорий
articles = articles_prep(articles)

CPU times: user 62.3 ms, sys: 10.7 ms, total: 73 ms
Wall time: 71.9 ms


### Собираем датафрейм для обучения модели 2го уровня

In [6]:
%%time
# добавим нужные признаки 
result = df_lvl2_prep(result)

CPU times: user 1min 47s, sys: 9.37 s, total: 1min 56s
Wall time: 1min 55s


In [7]:
%%time
# добавляем данные из датафремов покупателей и категорий
result = result.merge(customers, on='customer_id', how='left')
result = result.merge(articles, on='article_id', how='left')

CPU times: user 28.7 s, sys: 11.6 s, total: 40.3 s
Wall time: 40.3 s


In [8]:
del customers;
del articles;
gc.collect()

0

### Добавляем фичи
Добавляем следующие фичи:
1. сумма продаж по категориям товаров
2. средняя сумма покупок по категориям товаров
3. количество заказов покупателя с товарами категории
4. количество единиц товаров категории, приобретенное покупателем
5. сумма покупок покупателя по категории
6. средний чек покупателя по категории
7. общая сумма покупок покупателя
8. средний чек покупателя

In [9]:
%%time
result = get_features(result, transactions)

CPU times: user 4min 17s, sys: 2min 11s, total: 6min 29s
Wall time: 6min 42s


#### Выбираем фичи для модели

In [10]:
# все используемые фичи
features = list(result)
features.remove('target')
features.remove('customer_id')
features.remove('article_id')

# категориальные фичи
cat_feats = features.copy()
cat_feats.remove('age')
cat_feats.remove('customer_id_short')
cat_feats.remove('article_id_short')

In [11]:
# убираем из категариальных фичей новые численные
cat_feats.remove('articles_buy_sum')
cat_feats.remove('articles_buy_avg')

cat_feats.remove('customers_articles_num_sum')
cat_feats.remove('customers_articles_num_count')

cat_feats.remove('customers_articles_sum')
cat_feats.remove('customers_articles_avg')

cat_feats.remove('customers_spent_sum')
cat_feats.remove('customers_spent_avg')

### Построение моделей и получение топ12 рекомендаций
Построим следующие модели классификаторов:
- model 1 - покупатели до 24 лет. Молодые люди с невысокой покупательной способностью, несформировавшимися окончательно привычками, без сильного влияния детей.
- model 2 - покупатели 25-44 года. Платежеспособные люди с устоявшимися привычками и существенным влиянием детей на выбор товаров.
- model 3 - покупатели 45-64 года. Платежеспособные люди с устоявшимися привычками и уже без сильного влияния детей на выбор товаров.
- model 4 - покупатели старше 65 лет. Люди пенсионного возраста с устоявшимися привычками, но невысокой покупательной способностью.

### Предсказания моделей CatBoost и их оценка

In [12]:
%%time
customers_id, article_id, target, preds = get_lvl2_model_preds(result, features, cat_feats)

CPU times: user 5h 29min 47s, sys: 6min 26s, total: 5h 36min 13s
Wall time: 24min 17s


In [13]:
%%time
# построение датафрейма для получения оценок предсказаний модели
filename = 'result'
result_test = get_preds_result(customers_id, article_id, target, preds, filename)

CPU times: user 36.3 s, sys: 1.43 s, total: 37.7 s
Wall time: 37 s


In [14]:
print('Recall_own_rec_12 : ', result_test.apply(lambda row: recall(row['top_12_recs'], row['actual_article_id_short']), axis=1).mean())

Recall_own_rec_12 :  0.184025864929925


In [15]:
print('MAP_own_rec_12 : ', result_test.apply(lambda row: ap_k(row['top_12_recs'], row['actual_article_id_short'], 12), axis=1).mean())

MAP_own_rec_12 :  0.033345297338786654


### Предсказания моделей LGBM и их оценка

In [12]:
# преобразуем тип данных поля age для модели LGBM
result['age'] = result.apply(lambda row: col_convert_int(row['age']), axis=1)

In [13]:
%%time
customers_id, article_id, target, preds = get_lvl2_LGBM_model_preds(result, features, cat_feats)

Обучаем модель для возрастной группы 16-24
Обучаем модель для возрастной группы 25-44
Обучаем модель для возрастной группы 45-64
Обучаем модель для возрастной группы 65-96
CPU times: user 2h 43min 25s, sys: 1min 13s, total: 2h 44min 39s
Wall time: 12min 16s


In [14]:
%%time
# построение датафрейма для получения оценок предсказаний модели
filename = 'result'
result_test = get_preds_result(customers_id, article_id, target, preds, filename)

CPU times: user 35.1 s, sys: 1.38 s, total: 36.5 s
Wall time: 39.7 s


In [15]:
print('Recall_own_rec_12 : ', result_test.apply(lambda row: recall(row['top_12_recs'], row['actual_article_id_short']), axis=1).mean())

Recall_own_rec_12 :  0.1846590592006802


In [16]:
print('MAP_own_rec_12 : ', result_test.apply(lambda row: ap_k(row['top_12_recs'], row['actual_article_id_short'], 12), axis=1).mean())

MAP_own_rec_12 :  0.036161399028503224
